In [50]:
from google.colab import files
files.upload()

Saving algoparams_from_ui.json.json to algoparams_from_ui.json.json
Saving iris.csv to iris (1).csv


{'algoparams_from_ui.json.json': b'{\r\n  "session_name": "test",\r\n  "session_description": "test",\r\n  "design_state_data": {\r\n    "session_info": {\r\n      "project_id": "1",\r\n      "experiment_id": "kkkk-11",\r\n      "dataset": "iris_modified.csv",\r\n      "session_name": "test",\r\n      "session_description": "test"\r\n    },\r\n    "target": {\r\n      "prediction_type": "Regression",\r\n      "target": "petal_width",\r\n      "type": "regression",\r\n      "partitioning": true\r\n    },\r\n    "train": {\r\n      "policy": "Split the dataset",\r\n      "time_variable": "sepal_length",\r\n      "sampling_method": "No sampling(whole data)",\r\n      "split": "Randomly",\r\n      "k_fold": false,\r\n      "train_ratio": 0,\r\n      "random_seed": 0\r\n    },\r\n    "metrics": {\r\n      "optomize_model_hyperparameters_for": "AUC",\r\n      "optimize_threshold_for": "F1 Score",\r\n      "compute_lift_at": 0,\r\n      "cost_matrix_gain_for_true_prediction_true_result": 1,\r

In [51]:
import json
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import ExtraTreesRegressor, RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score

In [52]:
with open("algoparams_from_ui.json.json","r") as json_file:
  config=json.load(json_file)

df=pd.read_csv("iris.csv")

In [53]:
def parse_and_create_model(config,df):
  # Json file info
  session_info = config["design_state_data"]["session_info"]
  target_info = config["design_state_data"]["target"]
  feature_handling_info = config["design_state_data"]["feature_handling"]
  hyperparameters_info = config["design_state_data"]["hyperparameters"]
  algorithms_info = config["design_state_data"]["algorithms"]

  # Target and model prediction type
  targe=target_info["target"]
  type=target_info["type"]

  # Missing Value handling
  for feature_name, feature_info in feature_handling_info.items():
    feature_details = feature_info['feature_details']
    if 'missing_values' in feature_details and feature_details['missing_values'] == 'Impute':
        if feature_details.get('impute_with') == 'Average of values':
            df[feature_name].fillna(df[feature_name].mean(), inplace=True)
        elif feature_details.get('impute_with') == 'custom':
            impute_value = feature_details.get('impute_value')
            df[feature_name].fillna(impute_value, inplace=True)

  # Encoding
  categorical_feature = [feature_name for feature_name, feature_details in feature_handling_info.items() if feature_details["feature_variable_type"] == "text"]
  encoded_features = pd.get_dummies(df[categorical_feature], columns=categorical_feature, drop_first=True)

  df = pd.concat([df.drop(columns=categorical_feature), encoded_features], axis=1)

  # Feature Reduction Method
  feature_reduction_method = config["design_state_data"]["feature_reduction"]["feature_reduction_method"]
    # No reduction
  if feature_reduction_method == "No Reduction":
      pass
    # Correlation with target
  elif feature_reduction_method == "Corr with Target":
      threshold = 0.2
      correlation_matrix = df.corr()
      correlation_with_target = correlation_matrix[config["design_state_data"]["target"]["target"]]
      select_features = correlation_with_target[abs(correlation_with_target) >= threshold].index.tolist()
      df = df[select_features]
      # Tree Based
  elif feature_reduction_method == "Tree-based":
      x = df.drop(columns=config["design_state_data"]["target"]["target"])
      y = df[config["design_state_data"]["target"]["target"]]
      model = ExtraTreesRegressor()
      model.fit(x, y)
      importances = model.feature_importances_
      indices = np.argsort(importances)[::-1]
      features_to_keep = int(config["design_state_data"]["feature_reduction"]["num_of_features_to_keep"])
      selected_feature_indices = indices[:features_to_keep]
      selected_features = x.columns[selected_feature_indices]
      selected_features_with_target = selected_features.tolist() + [config["design_state_data"]["target"]["target"]]
      df = df[selected_features_with_target]
      # PCA
  elif feature_reduction_method == "PCA":
      num_of_components = int(config["design_state_data"]["feature_reduction"]["num_of_features_to_keep"])
      numerical_features = df.select_dtypes(include=[np.number]).drop(columns=config["design_state_data"]["target"]["target"]).columns

    # Apply PCA on numerical features
      pca = PCA(n_components=num_of_components)
      x_numerical = df[numerical_features]
      x_numerical_reduced = pca.fit_transform(x_numerical)
      df_numerical_reduced = pd.DataFrame(data=x_numerical_reduced, columns=[f"PCA_{i+1}" for i in range(num_of_components)])

      df = pd.concat([df_numerical_reduced, df[config["design_state_data"]["categorical_features"]], df[config["design_state_data"]["target"]["target"]]], axis=1)
  else:
      print("Unsupported feature reduction method:", feature_reduction_method)

  # Model Buiding
  prediction_models = {
      "Classification": {
          "RandomForestClassifier": RandomForestClassifier,
          "GBTClassifier": GradientBoostingClassifier,
          "LogisticRegression": LogisticRegression,
          "SVM": SVC,
          "DecisionTreeClassifier": DecisionTreeClassifier,
          "KNN": KNeighborsClassifier,
          "NeuralNetwork": MLPClassifier,
          },
      "Regression": {
          "RandomForestRegressor": RandomForestRegressor,
          "GBTRegressor": GradientBoostingRegressor,
          "LinearRegression": LinearRegression,
          "RidgeRegression": Ridge,
          "LassoRegression": Lasso,
          "ElasticNetRegression": ElasticNet,
          "xg_boost": XGBRegressor,
          "SGD": SGDRegressor,
          "DecisionTreeRegressor": DecisionTreeRegressor,
          "extra_random_trees": RandomForestRegressor,
          "KNN": KNeighborsRegressor,
          "neural_network": MLPRegressor,
          }
      }

  # Train - Test Split
  x = df.drop(columns=[config["design_state_data"]["target"]["target"]])
  y = df[config["design_state_data"]["target"]["target"]]
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=7)

  # Prediction Type
  prediction_type = config["design_state_data"]["target"]["prediction_type"]
  # applicable model
  applicable_models = prediction_models[prediction_type]

  models = {}
  for model_name, model_class in applicable_models.items():
      model_params = config["design_state_data"]["algorithms"][model_name]
      if model_params.get("is_selected", False):
          # Remove any unsupported keys from model_params
          supported_keys = model_class.__init__.__code__.co_varnames
          model_params = {k: v for k, v in model_params.items() if k in supported_keys}
          models[model_name] = model_class(**model_params)

  best_models = {}
  # Hyperparameter Tunning
  for model_name, model in models.items():
      hyperparameter = hyperparameters_info.get(model_name, {})
      grid_search = GridSearchCV(model, hyperparameter, cv=5, n_jobs=-1, verbose=2)
      grid_search.fit(x_train, y_train)

    # best model from the grid search
      best_model = grid_search.best_estimator_

    # best model and its parameters
      best_models[model_name] = {'model': best_model,
                                 'best_params': grid_search.best_params_}
  # Prediction
  y_pred = best_model.predict(x_test)

    # Evaluate the model using appropriate metrics based on the prediction type
  if prediction_type == "Classification":
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="weighted")
    recall = recall_score(y_test, y_pred, average="weighted")
    f1 = f1_score(y_test, y_pred, average="weighted")
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-score: {f1:.2f}")
  elif prediction_type == "Regression":
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Model: {model_name}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"R-squared (R2): {r2:.2f}")


parse_and_create_model(config,df)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Model: RandomForestRegressor
Mean Squared Error (MSE): 0.04
R-squared (R2): 0.92
